In [22]:
import numpy as np
import pandas as pd
import lxml
from bs4 import BeautifulSoup 
from urllib.request import urlopen
from selenium import webdriver
import time

<br>

<ul style="list-style-type:disc;">
  <li style = "font-family: Gulim; font-size:24px;line-height:1.4">
      <b> Scraping _ Selenium 사용<sup>1</sup> </b>
        <ol type = "A">
            <li style = "font-family: Gulim;font-size:16px;line-height:1.5"> 시간이 ,,,,, 오래 걸린다ㅠㅠ time.sleep () 을 낮춰도 좋을 것.</li>

In [24]:
# headless chrome 옵션 (셀레니움에서 visible 한 chrome 창을 띄우지 않는 것)
# 먼저 일반 으로 작업하여 chrome 이 정상작동 하는 것이 확인된후 add 한다.
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

chrome_driver_path = "C:/Sel/chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_path, options = options)
url = "https://www.istarbucks.co.kr/store/store_map.do"

address = []
for i in range(17):
    driver.get(url)
    time.sleep(0.5)
    
    loca = driver.find_element_by_class_name('loca_search')
    loca.click()
    time.sleep(0.5)

    loca = driver.find_element_by_class_name('sido_arae_box')
    li = loca.find_elements_by_tag_name('li')  # elements -> 복수
    li[i].click()
    time.sleep(0.5)
    
    if i != 16:
        gugun = driver.find_element_by_class_name('gugun_arae_box')
                # just _element_ -> 첫번째 li 만 가져옴 (전체)
        all_area = gugun.find_element_by_tag_name('li')
        all_area.click()
        time.sleep(1)

    source = driver.page_source
  
    bs = BeautifulSoup(source, 'lxml')
    ul_list = bs.find('ul',class_ = 'quickSearchResultBoxSidoGugun')
    li_list = ul_list.find_all('li')

    for k in li_list:
        address.append(k.find('p').text)


<br>

<ul style="list-style-type:disc;">
  <li style = "font-family: Gulim; font-size:24px;line-height:1.4">
      <b> 후처리_ 결과 dataframe화 및 시군구별 카운트  </b>
        <ol type = "A">
            <li style = "font-family: Gulim;font-size:16px;line-height:1.5">Starbucks 의 경우 text 가 '시도군구 포함 full 주소 + 전화번호' 로 이루어짐. 시도군구를 따로 떼어내기 위해 띄어쓰기에서 split(). 기본적으로는  split 결과의 index 0 + 1 이 시도군구이나, 세종특별자치시의 경우 군구가 없으므로 별개 처리. <br> <b>+ </b>  cartography의 ID를 만들기 위해 시도, 군구를 나눈다. 따라서 '안양SK  V1점' 와 세종특별자치시 (이 둘은 따로 처리) 제외 나머지는 나누어준다.</li>
<li style = "font-family: Gulim;font-size:16px;line-height:1.5"> draw_korea 와 호환하는 ID 만들기. 이 과정 중에 알 수 없는 이유로 인해 '군구' 데이터 중 일부가 float 형식으로 인식됨. 고로 applymap(str) 을 사용하여 전 데이터를 미리 string 으로 만든다. 시도 와 군구 를 나누면서 고성군 (강원, 경남) 는 각각 (강원), (경남) 로 구분. </li>
<li style = "font-family: Gulim;font-size:16px;line-height:1.5">draw_korea 에서는 거대 도시 (수원~부천 등 12개) 에서는 각각의 '자치구' 를 따로 정의, ID 로 만듦. 문제는 매장 주소는 일부는 자치구가 있으나 (ex. 부천시 원미구 상동) 일부는 도로명 주소여서인지 왜인지,,,ㅠㅠ 구가 없다 (ex. 부천시 부흥로 472). 그래서 이 부분은 후에 draw_korea의 ID에서 자치구를 없애고 모두 한 도시이름으로 변형하겠다.</li>
<li style = "font-family: Gulim;font-size:16px;line-height:1.5"> 전화번호의 경우 띄어쓰기가 된 경우도 있으나 안 된 경우도 있어, 추가 전처리가 필요. 현 코드에서는 진행하지 않음.</li>


In [79]:
df_address = pd.DataFrame(columns = ['시도', '군구','스벅_나머지데이터'])
n = 0
for ad in address :
    a = ad.split()
    
    if len(a[0]) >= 3:
        if a[0][2:] in ['시','광역시', '특별시', '도']:
            a[0] = a[0][:2] 
        elif a[0][3]  == '도':
            a[0] = a[0][0] + a[0][2]         
    df_address.loc[n, '시도'] = a[0]
    
    if a[1] == '고성군':
        if a[0] == '강원':
            df_address.loc[n, '군구'] = '고성(강원)'
        else:
            df_address.loc[n, '군구'] = '고성(경남)'  
    elif a[0] == '세종특별자치시':
        df_address.loc[n, '군구'] = '세종'
    else:
        if len(a[1]) >= 4:
            df_address.loc[n, '군구'] = a[1][:3]
        else: 
            df_address.loc[n , '군구'] = a[1]
    df_address.loc[n, '스벅_나머지데이터'] = ' '.join(a[2:])
    n += 1
len(df_address)

1308

In [80]:
df_address[600:610]

,시도,군구,스벅_나머지데이터
600,경기,시흥시,서해안로 699 (정왕동)031-8072-3638
601,경기,시흥시,중심상가2길 20-5 (정왕동)031-434-8550
602,경기,안산시,단원구 고잔로 72 (고잔동)031-405-3597
603,경기,안산시,단원구 적금로 120 (고잔동)031-414-3437
604,경기,안산시,단원구 원포공원1로 46 (초지동) 이마트031-405-9839
605,경기,안산시,"단원구 광덕대로 154, ,771-4 로데오타운1층 (고잔동)031-405-0678"
606,경기,안산시,"단원구 고잔로 102, 스타타워 1층 (고잔동)031-410-9658"
607,경기,안산시,단원구 중앙대로 907 (고잔동)031-439-8214
608,경기,안산시,단원구 중앙대로 397 (신길동)031-494-4507
609,경기,안성시,중앙로 308 (석정동)031-671-3610


In [81]:
# 만일을 대비해 df_address 내의 모든 데이터를 string 형식으로 전환
df_ID = df_address.applymap(str)
df_ID.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1308 entries, 0 to 1307
Data columns (total 3 columns):
시도           1308 non-null object
군구           1308 non-null object
스벅_나머지데이터    1308 non-null object
dtypes: object(3)
memory usage: 80.9+ KB


In [82]:
special_cities =  ['서울',  '인천',  '대전', '대구', '울산', '부산', '광주']
over3_gungu = ['고성(강원)', '고성(경남)','동두천', '의정부', '일산동', '남양주', '일산서', '서대문', '동대문', '영등포', '부산진', '해운대', '서귀포']

In [83]:
df_ID['ID'] = np.nan

n = len(df_ID)

for i in range(n):
    # 군구 중 원래 3음절이 아닌 경우 맨 끝의 '구' 또는 '시' 를 떼어낸다
    if df_ID.loc[i, '군구'] not in over3_gungu and len(str(df_ID.loc[i, '군구'])) >= 3:
        df_ID.loc[i,'군구'] = str(df_ID.loc[i, '군구'])[0:-1]
    # 시도 가 특별시, 광역시 등일 경우 draw_korea 와 호환을 위해 ID 에 포함한다.
    if df_ID.loc[i, '시도'] in special_cities:
        df_ID.loc[i, 'ID'] = df_ID.loc[i, '시도'] +' '+ str(df_ID.loc[i, '군구'])
    else:
        df_ID.loc[i, 'ID'] = df_ID.loc[i, '군구']

In [84]:
df_ID.to_csv("data_soomin/스벅_기본.csv",encoding='utf-8-sig')

In [85]:
df_ID[1000:1010]

,시도,군구,스벅_나머지데이터,ID
1000,부산,해운대,센텀2로 10 (우동) 센텀메디컬센타 1층 스타벅스 센텀로점051-742-6948,부산 해운대
1001,부산,해운대,센텀남대로 35 (우동) 센텀시티 신세계UEC051-745-1262,부산 해운대
1002,부산,해운대,달맞이길 57 (중동) 1~3층051-742-7802,부산 해운대
1003,부산,해운대,센텀중앙로 78 (우동)051-744-5741,부산 해운대
1004,부산,해운대,송정해변로 36 (송정동)051-702-9644,부산 해운대
1005,부산,해운대,해운대로 802 (좌동)051-702-6935,부산 해운대
1006,부산,해운대,"해운대해변로 257, 엘지하버타운 (우동)051-740-6595",부산 해운대
1007,부산,해운대,"APEC로 55, 벡스코내 (우동)051-742-2075",부산 해운대
1008,부산,해운대,해운대해변로 163 (우동)051-742-1655,부산 해운대
1009,부산,해운대,"해운대해변로298번길 24, 팔래드시즈 1-3호, 2-3호 (중동)051-743-2558",부산 해운대


In [86]:
df_count= pd.DataFrame(df_ID.groupby(df_ID.ID).count()['군구'])
df_count1 = df_count.rename(columns = {'군구':'스벅_빈도'})

In [87]:
df_count1.to_csv("data_soomin/스벅_시군구별카운트.csv",encoding='utf-8-sig')
df_count1.to_csv("data_soomin/스벅_시군구별카운트_euckr인코딩.csv")

---------
<p style = "font-family:Gulim; font-size:26px;font-weight:bold">참조 링크 </p>
<br>

1. [<문과의 파이썬> 셀레니움(Selenium)-(1)](https://brunch.co.kr/@jk-lab/18), JK의 계단 밑 연구실
